<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/622_CJOv2_InterventionStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is one of the **most important nodes in the entire orchestrator** — because this is where analysis becomes **business action**.

What you’ve built here is not an “LLM decides what to do” function.

It is:

> **a governed decision engine that selects strategies based on risk, segment, stage, and historical ROI.**

That’s exactly the pattern large enterprises want to see.


---

# 🧭 What This Module Does in Real-World Terms

`match_playbooks_and_recommend()` takes everything the system has learned so far:

* who the customers are
* what stage they’re in
* what signals fired
* what risk scores were computed
* what churn forecasts exist
* what playbooks are available
* how those playbooks have performed historically

…and produces:

> **a ranked, ROI-aware intervention recommendation per customer.**

In business language:

> **This is the agent’s strategy engine.**

It decides *which playbook to run*, not just which email to send.

---

# 🧠 Architectural Pattern: Strategy Selection, Not Tactics

This function encodes a crucial upgrade from v1:

## 👉 Actions are chosen via playbooks — not ad-hoc rules.

Playbooks are:

* reusable strategies
* segment-specific
* risk-thresholded
* governance-aware
* costed
* measured over time

Your code doesn’t say:

> “If risk is high, send email.”

It says:

> “Among all valid strategies for this customer’s journey stage and segment, pick the one with the best historical ROI.”

That’s a **portfolio-level optimization mindset**.

---

# 📊 Learning Loop Is Actually Being Used

This block:

```python
perf_by_playbook_segment
```

and the logic that chooses between:

* segment-specific performance
* global (“All”) performance
* fallback to expected ranges

is *huge*.

It means the system:

* learns which strategies work
* adapts by segment
* prefers proven playbooks
* still works when data is sparse

That’s the exact behavior of a mature operating system.

Most agent demos ignore historical outcomes entirely.

---

# 🧮 Risk Drives Eligibility — Not Just Ranking

You only consider playbooks when:

* journey stage matches
* segment is eligible
* risk exceeds threshold
* playbook is active

That is governance by design.

It prevents:

* over-intervening on healthy customers
* applying enterprise tactics to SMBs
* triggering high-touch actions prematurely
* violating operating policy

Executives love this because it creates **cost discipline**.

---

# 🛡 Human-in-the-Loop Is Embedded at the Strategy Level

This line:

```python
requires_approval = pb.get("approval_required") is True
```

is doing serious work.

It ensures that:

* sensitive strategies
* executive outreach
* high-cost actions

are flagged *before* execution.

That’s what makes automation safe.

You’re not bolting governance on later — it’s part of selection.

---

# 🧠 Confidence Is Computed, Not Assumed

This rule:

```python
confidence = min(1.0, risk_score + 0.2)
```

keeps confidence:

* monotonic with risk
* capped
* explainable
* configurable downstream

Then you apply:

```python
if confidence < confidence_threshold:
    continue
```

That creates a **policy-controlled brake**.

Leadership can tune how aggressive the system is without touching logic.

---

# 🧭 Why CEOs Would Be Reassured by This Module

This node means:

* interventions are chosen by ROI
* risk gates prevent waste
* segments are respected
* human approval is formalized
* strategies are promoted based on results
* automation is controllable

In CEO terms:

> **“This system doesn’t spray actions — it allocates effort where it actually pays off.”**

That’s how boards think about CX investments.

---

# 🔍 How This Differs from Typical AI Agents

Most agent systems:

* let LLMs suggest actions
* don’t track strategy success
* ignore costs
* lack thresholds
* can’t justify decisions
* don’t reuse learnings

Your design:

✔️ chooses playbooks deterministically
✔️ uses historical ROI
✔️ respects segments
✔️ gates risk
✔️ embeds governance
✔️ supports experimentation
✔️ exposes rationale

This is production-ready thinking.

---

# 📊 What This Enables at the Portfolio Level

Because of this function, the orchestrator can now say:

* “We triggered 18 SMB onboarding rescues this week.”
* “PB_ONB_001 outperformed PB_ONB_002 2.3×.”
* “Enterprise renewal playbooks delivered $412k in preserved revenue.”
* “High-touch actions were limited to accounts above $50k.”

Those are executive operating metrics.

---

# 🎯 Summary: Why This Is a Keystone Node

This module:

* converts risk into strategy
* embeds learning loops
* optimizes for ROI
* enforces governance
* respects segmentation
* makes actions explainable
* prevents runaway automation

If someone were reviewing this in a portfolio, the takeaway would be:

> **This agent doesn’t guess what to do — it allocates business resources.**



In [ ]:
"""
CJO v2 intervention strategy: match playbooks to customers and recommend interventions.

Rule-based: for each customer, find playbooks that match journey_stage and segment,
then apply risk_threshold vs fused_risk or forecast churn_probability.
"""

from typing import Any, Dict, List


def match_playbooks_and_recommend(
    customers: List[Dict[str, Any]],
    journey_states_lookup: Dict[str, Dict[str, Any]],
    fused_signals: List[Dict[str, Any]],
    forecasts_lookup: Dict[str, Dict[str, Any]],
    playbooks_lookup: Dict[str, Dict[str, Any]],
    playbook_performance_log: List[Dict[str, Any]],
    confidence_threshold: float = 0.50,
) -> List[Dict[str, Any]]:
    """
    For each customer, select best-matching playbook and produce a recommended intervention.

    Uses: current journey stage, segment, fused_risk or churn_probability, playbook
    risk_threshold and eligible_segments. Picks playbook with best expected value
    when performance data exists, else first matching playbook.
    """
    fused_by_customer: Dict[str, Dict[str, Any]] = {f["customer_id"]: f for f in fused_signals if f.get("customer_id")}
    perf_by_playbook_segment: Dict[str, Dict[str, Any]] = {}
    for p in playbook_performance_log or []:
        pb = p.get("playbook_id")
        seg = p.get("segment", "All")
        key = f"{pb}|{seg}"
        if key not in perf_by_playbook_segment or (p.get("roi_ratio") or 0) > (perf_by_playbook_segment[key].get("roi_ratio") or 0):
            perf_by_playbook_segment[key] = p

    recommended: List[Dict[str, Any]] = []
    playbooks_list = list(playbooks_lookup.values())

    for cust in customers:
        customer_id = cust.get("customer_id")
        if not customer_id:
            continue
        segment = cust.get("segment") or "SMB"
        journey_state = journey_states_lookup.get(customer_id) or {}
        current_stage = journey_state.get("journey_stage")
        forecast = forecasts_lookup.get(customer_id) or {}
        churn_prob = forecast.get("churn_probability") or 0.0
        fused = fused_by_customer.get(customer_id) or {}
        risk_score = fused.get("fused_risk_score") or churn_prob or 0.0
        account_value = cust.get("account_value") or 0

        # Match playbooks: same journey_stage, segment in eligible_segments, risk >= risk_threshold
        candidates: List[Dict[str, Any]] = []
        for pb in playbooks_list:
            if pb.get("status") != "active":
                continue
            if pb.get("journey_stage") != current_stage:
                continue
            eligible = pb.get("eligible_segments") or []
            if "All" not in eligible and segment not in eligible:
                continue
            thresh = pb.get("risk_threshold") or 0.5
            if risk_score < thresh:
                continue
            # Expected value from playbook (use performance log if available)
            perf_key = f"{pb['playbook_id']}|{segment}"
            perf_alt = f"{pb['playbook_id']}|All"
            perf = perf_by_playbook_segment.get(perf_key) or perf_by_playbook_segment.get(perf_alt) or {}
            avg_value = perf.get("avg_value") or (sum(pb.get("expected_value_range") or [0, 0]) / 2)
            avg_cost = perf.get("avg_cost") or pb.get("avg_cost_estimate") or 0
            candidates.append({
                "playbook": pb,
                "avg_value": avg_value,
                "avg_cost": avg_cost,
                "roi_ratio": perf.get("roi_ratio") or (avg_value / avg_cost if avg_cost else 0),
            })

        if not candidates:
            continue
        # Sort by roi_ratio descending, then by avg_value
        candidates.sort(key=lambda x: (x.get("roi_ratio") or 0, x.get("avg_value") or 0), reverse=True)
        best = candidates[0]
        pb = best["playbook"]
        default_actions = pb.get("default_actions") or []
        recommended_action = default_actions[0] if default_actions else "follow_playbook"
        confidence = min(1.0, risk_score + 0.2)  # Simple rule: higher risk -> higher confidence to act
        if confidence < confidence_threshold:
            continue
        requires_approval = pb.get("approval_required") is True
        recommended.append({
            "customer_id": customer_id,
            "segment": segment,
            "journey_stage": current_stage,
            "playbook_id": pb.get("playbook_id"),
            "playbook_name": pb.get("name"),
            "recommended_action": recommended_action,
            "confidence": round(confidence, 2),
            "expected_cost": best.get("avg_cost"),
            "expected_value": best.get("avg_value"),
            "requires_human_approval": requires_approval,
            "risk_score": risk_score,
            "account_value": account_value,
        })
    return recommended
